In [ ]:
%cd ..

import torch
import os
import json
import time
import random
import hashlib
import gradio as gr
from PIL import Image
from helpers.models.hidream.pipeline import HiDreamImagePipeline
from helpers.models.hidream.transformer import HiDreamImageTransformer2DModel
from transformers import PreTrainedTokenizerFast, LlamaForCausalLM
from lycoris import create_lycoris_from_weights

# Load base model components
llama_repo = "/local/yada/apps/HiDream-I1-a/Meta-Llama-3.1-8B-Instruct"
model_id = "HiDream-ai/HiDream-I1-Full"

# --- LoRA setup ---
lora_options = {
    "anime 400 run4": "/local/yada/apps/SimpleTuner-a/output/models-hidream-run04/pytorch_lora_weights.safetensors",
}

save_dir = "output/gradio_3".rstrip('/')

tokenizer_4 = PreTrainedTokenizerFast.from_pretrained(llama_repo)
text_encoder_4 = LlamaForCausalLM.from_pretrained(llama_repo, output_hidden_states=True, torch_dtype=torch.bfloat16)
transformer = HiDreamImageTransformer2DModel.from_pretrained(model_id, torch_dtype=torch.bfloat16, subfolder="transformer")

pipeline = HiDreamImagePipeline.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    tokenizer_4=tokenizer_4,
    text_encoder_4=text_encoder_4,
    transformer=transformer,
)

pipeline.to('cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu')


current_lora_path = None

def apply_lora_if_needed(new_lora_path, lora_scale=1.0):
    global current_lora_path
    if current_lora_path == new_lora_path:
        return  # No need to reapply

    wrapper, _ = create_lycoris_from_weights(lora_scale, new_lora_path, pipeline.transformer)
    wrapper.merge_to()
    current_lora_path = new_lora_path

# --- Image Generation ---
def generate_image(prompt, negative_prompt, width, height, seed, guidance_scale, num_inference_steps, lora_choice):
    lora_path = lora_options[lora_choice]
    apply_lora_if_needed(lora_path)

    if seed == -1:
        seed = random.randint(0, 2**32 - 1)

    # Encode prompts
    t5_embeds, llama_embeds, negative_t5_embeds, negative_llama_embeds, pooled_embeds, negative_pooled_embeds = pipeline.encode_prompt(
        prompt=prompt, prompt_2=prompt, prompt_3=prompt, prompt_4=prompt, num_images_per_prompt=1
    )

    # Generate
    device = 'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'
    image = pipeline(
        t5_prompt_embeds=t5_embeds,
        llama_prompt_embeds=llama_embeds,
        pooled_prompt_embeds=pooled_embeds,
        negative_t5_prompt_embeds=negative_t5_embeds,
        negative_llama_prompt_embeds=negative_llama_embeds,
        negative_pooled_prompt_embeds=negative_pooled_embeds,
        num_inference_steps=num_inference_steps,
        generator=torch.Generator(device=device).manual_seed(seed),
        width=width,
        height=height,
        guidance_scale=guidance_scale,
    ).images[0]

    # Save with metadata
    timestamp = int(time.time())
    image_hash = hashlib.sha256(f"{prompt}{negative_prompt}{timestamp}".encode()).hexdigest()[:10]
    base_path = f"{save_dir}/{timestamp}_{image_hash}"
    os.makedirs(save_dir, exist_ok=True)
    image.save(f"{base_path}.png")

    metadata = {
        "prompt": prompt,
        "negative_prompt": negative_prompt,
        "width": width,
        "height": height,
        "timestamp": timestamp,
        # "image_path": f"{base_path}.png",
        "seed": seed,
        "lora": lora_path,
        "guidance_scale": guidance_scale,
        "num_inference_steps": num_inference_steps,
    }
    with open(f"{base_path}.json", "w") as f:
        json.dump(metadata, f, indent=4)

    return image

# --- Gradio Interface ---
demo = gr.Interface(
    fn=generate_image,
    inputs=[
        gr.Textbox(label="Prompt", lines=4, value="an anime illustration of hatsune miku, 1girl, white gloves, dress, cat ears, maid outfit, indoors, cat ears"),
        gr.Textbox(label="Negative Prompt", lines=2, value="ugly, cropped, blurry, low-quality, mediocre average"),
        gr.Slider(minimum=256, maximum=1536, step=64, value=768, label="Width"),
        gr.Slider(minimum=256, maximum=1536, step=64, value=1280, label="Height"),
        gr.Slider(minimum=-1, maximum=2**32 - 1, step=1, value=-1, label="Seed"),
        gr.Slider(minimum=0.1, maximum=20.0, step=0.1, value=3.2, label="Guidance Scale"),
        gr.Slider(minimum=1, maximum=200, step=1, value=50, label="Inference Steps"),
        gr.Dropdown(choices=list(lora_options.keys()), label="LoRA Weights", value=list(lora_options.keys())[0]),
    ],
    outputs=gr.Image(label="Generated Image"),
    title="Cat Maid Generator: LoRA Swap Edition",
    description=f"Now with dynamic LoRA switching. Currently using base model: `{model_id}`.",
)

if __name__ == "__main__":
    demo.launch(share=True)

/root/miniconda3/envs/simpletuner/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


/local/yada/apps/SimpleTuner-a


[2025-04-14 15:49:52,968] [INFO] [real_accelerator.py:222:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/root/miniconda3/envs/simpletuner/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/root/miniconda3/envs/simpletuner/compiler_compat/ld: warning: librt.so.1, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/root/miniconda3/envs/simpletuner/compiler_compat/ld: warning: libpthread.so.0, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/root/miniconda3/envs/simpletuner/compiler_compat/ld: warning: libstdc++.so.6, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/root/miniconda3/envs/simpletuner/compiler_compat/ld: warning: libm.so.6, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/root/miniconda3/envs/simpletuner/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `std::runtime_error::~runtime_error()@GLIBCXX_3.4'
/root/miniconda3/envs/simp

HiDream: FlashAttention3 not found, trying FlashAttention2
HiDream: FlashAttention2 failed to load (No module named 'flash_attn'), falling back to Torch SDPA.


...

2025-04-14 15:51:45|[LyCORIS]-INFO: Loading Modules from state dict...
2025-04-14 15:51:45|[LyCORIS]-INFO: 256 Modules Loaded


`torch.nn.functional.scaled_dot_product_attention` does not support `output_attentions=True`. Falling back to eager attention. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.


  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

: 